In [97]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json

from selenium.webdriver.support.ui import Select, WebDriverWait
img_path = 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/'
def get_champion_data(url):
    driver = webdriver.Chrome(r'C:\Users\Sibtain\Downloads\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    champ_url = driver.current_url.split('championstats')[0]
    name = find_between(champ_url, "https://lolcounter.gg/", "/")
    folder = 'champstats/'+name
    print("starting ", name)
    select = driver.find_elements_by_tag_name("fieldset")  #get the list of leagues & the list of lanes played
    league_options = select[0].find_elements_by_tag_name("li") #get all list elements inside leagues (platplus, plat, etc.)
    lane_options = select[1].find_elements_by_tag_name("li") #get all list elements inside lanes (mid, top, etc.)

    optionsList = {'lane': [], 'league': []}

    for option in league_options: #iterate over the options, place attribute of onclick in list
        optionsList['league'].append(option.get_attribute("onclick"))

    for option in lane_options: #iterate over the options, place attribute of onclick in list
        optionsList['lane'].append(option.get_attribute("onclick"))

    championJson = {}
    for optionHref in optionsList['league']:
        optionValue = 'platplus'
        if "championstats.php?" not in optionHref:
            optionValue = find_between(optionHref, "championstats", ".php?")
            link = champ_url + find_between(optionHref, "location.href='", "';")
            print(link)
            driver.get(link)
        for laneHref in optionsList['lane']:
            lane = find_between(laneHref, "Lane=", "';")
            link = champ_url + find_between(optionHref, "location.href='", "?") + find_between(laneHref, "hp", "';")
            print(link)
            driver.get(link)
            soup = BeautifulSoup(driver.page_source,'lxml')

            print("starting loop on option {} in lane {}".format(optionValue, lane))
            #build order
            build_order = driver.find_element_by_class_name("build-wrapper")
            build_order_imgs = build_order.find_elements_by_tag_name("img")
            build = []
            for img in build_order_imgs:
                #get url and id
                x = {}
                x["src"] = img.get_attribute("src")
                x["id"] = find_between(x["src"], "/item/", ".png")
                build.append(x)
            
            skill_order = driver.find_element_by_class_name("Skill-Order2")    #get highest win % skill order
            skill_order_table = skill_order.find_element_by_class_name("skill-order")
            skills = skill_order_table.find_elements_by_class_name("skill")
            counter = -1
            skill_images = []
            skill_order = [0] * 18
            for skill in skills:
                if counter == -1:
                    counter += 1
                    continue
                # skill has img and skill selections
                img = skill.find_element_by_tag_name("img") #deal with img first
                img_src = img.get_attribute("src")
                skill_images.append(img_src)
                skills_selection = skill.find_element_by_class_name("skill-selections")
                skills_selection_list = skills_selection.find_elements_by_tag_name("div")
                skill_counter = 0
                current_skill = 'qwer'[counter]
                for skill_selection in skills_selection_list:
                    text = skill_selection.text.replace(" ", "")
                    if text:
                        skill_order[skill_counter] = current_skill
                    skill_counter += 1
                counter += 1
            if skill_order == [0] * 18:
                continue
            if optionValue not in championJson:
                championJson[optionValue] = {}
            if lane not in championJson[optionValue]:
                championJson[optionValue][lane] = {}
            if "skillImages" not in championJson:
                championJson["skillImages"] = skill_images
            championJson[optionValue][lane]["skill_order"] = skill_order

            table = soup.find_all('table')[0] # get stats table
            df = pd.read_html(str(table))[0] # add that to df
            stats_list = json.loads(df.to_json(orient='records'))
            stats_dict = {}
            for stat in stats_list:
                type_stat = stat["Type"]
                ave = stat["Average"]
                stat.clear()
                stats_dict[type_stat] = ave
            championJson[optionValue][lane]["stats"] = stats_dict
            championJson[optionValue][lane]["build"] = build
        #get counters / lane
        link = champ_url
        if optionValue != 'platplus':
            link = champ_url + 'counterstats' + optionValue + '.php'
        #print(link)
        print("getting counter info per league per lane")
        driver.get(link)
        lanes = driver.find_elements_by_class_name("champbox1-subwarpper")
        for lane_elem in lanes:
            lane_with_name = lane_elem.find_element_by_tag_name("h1").text
            if lane_with_name is not None:
                lane = lane_with_name.lower().split(" ")
                #print(lane)
                lane = lane[len(lane) - 1]
                champs = {'weak': [], 'strong': []}
                champ_sections = [lane_elem.find_element_by_class_name("champ-blockwrapper1"), lane_elem.find_element_by_class_name("champ-blockwrapper2")]
                for section in champ_sections:
                    champ_blocks = section.find_elements_by_class_name("champ-block")
                    weak_or_strong = section.find_element_by_tag_name("div").text
                    if 'weak' in weak_or_strong.lower():
                        key = 'weak'
                    else:
                        key = 'strong'
                    for i in range(0, min(5, len(champ_blocks))):
                        champ = champ_blocks[i]
                        x = {}
                        x['name'] = champ.find_element_by_class_name("name").text
                        x['win_rate'] = champ.find_element_by_class_name("per-bar").text
                        style = champ.find_element_by_class_name("left").get_attribute('style')
                        x['img'] = img_path + find_between(style,"champion/", ".png") + ".png"
                        champs[key].append(x)  
                #print(champs)
                if lane == "mid":
                    lane = "MIDDLE"
                if lane.upper() in championJson[optionValue]:
                    championJson[optionValue][lane.upper()]["counters"] = champs
            
    driver.close()
    filename = folder + '.json'
    with open(filename, 'w') as f:
         json.dump(championJson, f)
    print("success! ", name)

In [94]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [98]:
def get_champ_links():
    links = []
    url = 'https://lolcounter.gg/index.php'
    driver = webdriver.Chrome(r'C:\Users\Sibtain\Downloads\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    host = "https://lolcounter.gg/"
    champ_stats = '/championstats.php?Lane=TOP'
    champion_table = driver.find_element_by_id("champion-list")
    elements_list = champion_table.find_elements_by_class_name("inlinechamps")
    for champ in elements_list:
        name = champ.get_attribute("data-champion-name").replace(" ", "%20")
        link = host + name + champ_stats
        links.append(link)
    driver.close()
    return links
champ_links = get_champ_links()
x = 0
for link in champ_links[32:]:
    print(str(x) + " " + link)
    x += 1
    get_champion_data(link)

0 https://lolcounter.gg/garen/championstats.php?Lane=TOP
starting  garen
https://lolcounter.gg/garen/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
getting counter info per league per lane
https://lolcounter.gg/garen/championstatsplat.php?Lane=TOP
https://lolcounter.gg/garen/championstatsplat.php?Lane=TOP
starting loop on option plat in lane TOP
getting counter info per league per lane
https://lolcounter.gg/garen/championstatsgold.php?Lane=TOP
https://lolcounter.gg/garen/championstatsgold.php?Lane=TOP
starting loop on option gold in lane TOP
getting counter info per league per lane
https://lolcounter.gg/garen/championstatssilver.php?Lane=TOP
https://lolcounter.gg/garen/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
getting counter info per league per lane
https://lolcounter.gg/garen/championstatsbronze.php?Lane=TOP
https://lolcounter.gg/garen/championstatsbronze.php?Lane=TOP
starting loop on option bronze in lane TOP
getting counter

https://lolcounter.gg/illaoi/championstatssilver.php?Lane=TOP
https://lolcounter.gg/illaoi/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
getting counter info per league per lane
https://lolcounter.gg/illaoi/championstatsbronze.php?Lane=TOP
https://lolcounter.gg/illaoi/championstatsbronze.php?Lane=TOP
starting loop on option bronze in lane TOP
getting counter info per league per lane
success!  illaoi
7 https://lolcounter.gg/irelia/championstats.php?Lane=TOP
starting  irelia
https://lolcounter.gg/irelia/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
getting counter info per league per lane
https://lolcounter.gg/irelia/championstatsplat.php?Lane=TOP
https://lolcounter.gg/irelia/championstatsplat.php?Lane=TOP
starting loop on option plat in lane TOP
getting counter info per league per lane
https://lolcounter.gg/irelia/championstatsgold.php?Lane=TOP
https://lolcounter.gg/irelia/championstatsgold.php?Lane=TOP
starting loop on option gold

https://lolcounter.gg/jayce/championstatsbronze.php?Lane=TOP
starting loop on option bronze in lane TOP
https://lolcounter.gg/jayce/championstatsbronze.php?Lane=MIDDLE
starting loop on option bronze in lane MIDDLE
getting counter info per league per lane
success!  jayce
13 https://lolcounter.gg/jhin/championstats.php?Lane=TOP
starting  jhin
https://lolcounter.gg/jhin/championstats.php?Lane=ADC
starting loop on option platplus in lane ADC
getting counter info per league per lane
https://lolcounter.gg/jhin/championstatsplat.php?Lane=ADC
https://lolcounter.gg/jhin/championstatsplat.php?Lane=ADC
starting loop on option plat in lane ADC
getting counter info per league per lane
https://lolcounter.gg/jhin/championstatsgold.php?Lane=ADC
https://lolcounter.gg/jhin/championstatsgold.php?Lane=ADC
starting loop on option gold in lane ADC
getting counter info per league per lane
https://lolcounter.gg/jhin/championstatssilver.php?Lane=ADC
https://lolcounter.gg/jhin/championstatssilver.php?Lane=ADC
s

https://lolcounter.gg/katarina/championstatsgold.php?Lane=MIDDLE
https://lolcounter.gg/katarina/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
getting counter info per league per lane
https://lolcounter.gg/katarina/championstatssilver.php?Lane=MIDDLE
https://lolcounter.gg/katarina/championstatssilver.php?Lane=MIDDLE
starting loop on option silver in lane MIDDLE
getting counter info per league per lane
https://lolcounter.gg/katarina/championstatsbronze.php?Lane=MIDDLE
https://lolcounter.gg/katarina/championstatsbronze.php?Lane=MIDDLE
starting loop on option bronze in lane MIDDLE
getting counter info per league per lane
success!  katarina
20 https://lolcounter.gg/kayle/championstats.php?Lane=TOP
starting  kayle
https://lolcounter.gg/kayle/championstats.php?Lane=TOP
starting loop on option platplus in lane TOP
https://lolcounter.gg/kayle/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
getting counter info per league per lane
htt

starting  kogmaw
https://lolcounter.gg/kogmaw/championstats.php?Lane=ADC
starting loop on option platplus in lane ADC
getting counter info per league per lane
https://lolcounter.gg/kogmaw/championstatsplat.php?Lane=ADC
https://lolcounter.gg/kogmaw/championstatsplat.php?Lane=ADC
starting loop on option plat in lane ADC
getting counter info per league per lane
https://lolcounter.gg/kogmaw/championstatsgold.php?Lane=ADC
https://lolcounter.gg/kogmaw/championstatsgold.php?Lane=ADC
starting loop on option gold in lane ADC
getting counter info per league per lane
https://lolcounter.gg/kogmaw/championstatssilver.php?Lane=ADC
https://lolcounter.gg/kogmaw/championstatssilver.php?Lane=ADC
starting loop on option silver in lane ADC
getting counter info per league per lane
https://lolcounter.gg/kogmaw/championstatsbronze.php?Lane=ADC
https://lolcounter.gg/kogmaw/championstatsbronze.php?Lane=ADC
starting loop on option bronze in lane ADC
getting counter info per league per lane
success!  kogmaw
27 h

...

https://lolcounter.gg/teemo/championstatsplat.php?Lane=TOP
starting loop on option plat in lane TOP
getting counter info per league per lane
https://lolcounter.gg/teemo/championstatsgold.php?Lane=TOP
https://lolcounter.gg/teemo/championstatsgold.php?Lane=TOP
starting loop on option gold in lane TOP
getting counter info per league per lane
https://lolcounter.gg/teemo/championstatssilver.php?Lane=TOP
https://lolcounter.gg/teemo/championstatssilver.php?Lane=TOP
starting loop on option silver in lane TOP
getting counter info per league per lane
https://lolcounter.gg/teemo/championstatsbronze.php?Lane=TOP
https://lolcounter.gg/teemo/championstatsbronze.php?Lane=TOP
starting loop on option bronze in lane TOP
getting counter info per league per lane
success!  teemo
78 https://lolcounter.gg/thresh/championstats.php?Lane=TOP
starting  thresh
https://lolcounter.gg/thresh/championstats.php?Lane=SUPPORT
starting loop on option platplus in lane SUPPORT
getting counter info per league per lane
https

https://lolcounter.gg/twitch/championstatsplat.php?Lane=ADC
https://lolcounter.gg/twitch/championstatsplat.php?Lane=ADC
starting loop on option plat in lane ADC
getting counter info per league per lane
https://lolcounter.gg/twitch/championstatsgold.php?Lane=ADC
https://lolcounter.gg/twitch/championstatsgold.php?Lane=ADC
starting loop on option gold in lane ADC
getting counter info per league per lane
https://lolcounter.gg/twitch/championstatssilver.php?Lane=ADC
https://lolcounter.gg/twitch/championstatssilver.php?Lane=ADC
starting loop on option silver in lane ADC
getting counter info per league per lane
https://lolcounter.gg/twitch/championstatsbronze.php?Lane=ADC
https://lolcounter.gg/twitch/championstatsbronze.php?Lane=ADC
starting loop on option bronze in lane ADC
getting counter info per league per lane
success!  twitch
84 https://lolcounter.gg/udyr/championstats.php?Lane=TOP
starting  udyr
https://lolcounter.gg/udyr/championstats.php?Lane=JUNGLE
starting loop on option platplus i

starting loop on option bronze in lane SUPPORT
getting counter info per league per lane
success!  velkoz
90 https://lolcounter.gg/vi/championstats.php?Lane=TOP
starting  vi
https://lolcounter.gg/vi/championstats.php?Lane=JUNGLE
starting loop on option platplus in lane JUNGLE
getting counter info per league per lane
https://lolcounter.gg/vi/championstatsplat.php?Lane=JUNGLE
https://lolcounter.gg/vi/championstatsplat.php?Lane=JUNGLE
starting loop on option plat in lane JUNGLE
getting counter info per league per lane
https://lolcounter.gg/vi/championstatsgold.php?Lane=JUNGLE
https://lolcounter.gg/vi/championstatsgold.php?Lane=JUNGLE
starting loop on option gold in lane JUNGLE
getting counter info per league per lane
https://lolcounter.gg/vi/championstatssilver.php?Lane=JUNGLE
https://lolcounter.gg/vi/championstatssilver.php?Lane=JUNGLE
starting loop on option silver in lane JUNGLE
getting counter info per league per lane
https://lolcounter.gg/vi/championstatsbronze.php?Lane=JUNGLE
https:/

success!  wukong
96 https://lolcounter.gg/xayah/championstats.php?Lane=TOP
starting  xayah
https://lolcounter.gg/xayah/championstats.php?Lane=ADC
starting loop on option platplus in lane ADC
getting counter info per league per lane
https://lolcounter.gg/xayah/championstatsplat.php?Lane=ADC
https://lolcounter.gg/xayah/championstatsplat.php?Lane=ADC
starting loop on option plat in lane ADC
getting counter info per league per lane
https://lolcounter.gg/xayah/championstatsgold.php?Lane=ADC
https://lolcounter.gg/xayah/championstatsgold.php?Lane=ADC
starting loop on option gold in lane ADC
getting counter info per league per lane
https://lolcounter.gg/xayah/championstatssilver.php?Lane=ADC
https://lolcounter.gg/xayah/championstatssilver.php?Lane=ADC
starting loop on option silver in lane ADC
getting counter info per league per lane
https://lolcounter.gg/xayah/championstatsbronze.php?Lane=ADC
https://lolcounter.gg/xayah/championstatsbronze.php?Lane=ADC
starting loop on option bronze in lane A

https://lolcounter.gg/zed/championstatssilver.php?Lane=MIDDLE
starting loop on option silver in lane MIDDLE
getting counter info per league per lane
https://lolcounter.gg/zed/championstatsbronze.php?Lane=MIDDLE
https://lolcounter.gg/zed/championstatsbronze.php?Lane=MIDDLE
starting loop on option bronze in lane MIDDLE
getting counter info per league per lane
success!  zed
103 https://lolcounter.gg/ziggs/championstats.php?Lane=TOP
starting  ziggs
https://lolcounter.gg/ziggs/championstats.php?Lane=MIDDLE
starting loop on option platplus in lane MIDDLE
getting counter info per league per lane
https://lolcounter.gg/ziggs/championstatsplat.php?Lane=MIDDLE
https://lolcounter.gg/ziggs/championstatsplat.php?Lane=MIDDLE
starting loop on option plat in lane MIDDLE
getting counter info per league per lane
https://lolcounter.gg/ziggs/championstatsgold.php?Lane=MIDDLE
https://lolcounter.gg/ziggs/championstatsgold.php?Lane=MIDDLE
starting loop on option gold in lane MIDDLE
getting counter info per le

In [66]:
min(5, len([]))

0

In [72]:
x = "Vayne Adc".split(" ")
x[len(x) - 1]

'Adc'

In [81]:
if "asd" in {"asd":1}:
    print("asd")

asd
